<a href="https://colab.research.google.com/github/rwurdig/TESTEPYSPARK-Confitec/blob/main/TESTEPYSPARK_Confitec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 200 kB 35.9 MB/s 
     |████████████████████████████████| 199 kB 36.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=d3e450a1c7208a05f7439c6c32a79173a84cdeec356ebe0c7ff6bba9f8574a7d
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [173]:
# Instalando Bibliotecas

from pyspark.sql import SparkSession
from  pyspark.sql.functions import *
from pyspark.sql import types
from pyspark.sql.functions import when, col
from pyspark.sql.functions import udf, to_timestamp, lower, regexp_replace, lit, concat, col
from pyspark.sql.types import TimestampType
from pyspark.sql.types import *
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import os



In [ ]:
# Setup

In [167]:


#Definicacao do Spark Session
spark = SparkSession \
    .builder \
    .appName("Confitec") \
    .enableHiveSupport() \
    .config("spark.executor.heartbeatInterval","8000") \
    .getOrCreate()
	
 

In [168]:
#Leitura do arquivo Parquet

df = spark.read.parquet("OriginaisNetflix.parquet")
df.printSchema()
df.first().asDict()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)



{'Title': 'House of Cards',
 'Genre': 'Political drama',
 'GenreLabels': 'political,drama',
 'Premiere': '1-Feb-13',
 'Seasons': '6 seasons, 73 episodes',
 'SeasonsParsed': 6,
 'EpisodesParsed': 73,
 'Length': '42–59 min.',
 'MinLength': 42,
 'MaxLength': 59,
 'Status': 'Ended',
 'Active': 0,
 'Table': 'Drama',
 'Language': 'English',
 'dt_inclusao': '2021-03-16T21:20:24.167-03:00'}

In [ ]:
# Questão 01 - Transformar os campos "Premiere" e "dt_inclusao" de string para datetime.

In [170]:
df = df.withColumn('mes', substring('Premiere', -6,3)).withColumn('ano', substring('Premiere', -2,3)).withColumn('dia', substring('Premiere', 1,2))
df = df.withColumn("dia", regexp_replace(col("dia"), """-""", ""))

meses = {'Jan': '01',
         'Feb': '02',
         'Mar': '03',
         'Apr': '04',
         'May': '05',
         'Jun': '06',
         'Jul': '07',
         'Aug': '08',
         'Sep': '09',
         'Oct': '10',
         'Nov': '11',
         'Dec': '12'}

df = df.replace(meses,subset=["mes"])
df  = df.withColumn("Premiere",concat(lit("20"),col("ano"),lit("-"),col("mes"),lit("-"),col("dia")))

df = df.withColumn("Premiere",date_format('Premiere', 'yyy-MM-dd'))
df.withColumn("dt_inclusao",to_date(col("dt_inclusao")))
df =  df.withColumn("Premiere",to_date(col("dt_inclusao")))

In [171]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: date (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- dia: string (nullable = true)



In [174]:
# Questão 02 - Ordenar os dados por ativos e gênero de forma decrescente, 0 = inativo e 1 = ativo, todos com número 1 devem aparecer primeiro.

df_02 = df.sort(col("Active").desc(),col("Genre").asc())
df_02.show()

+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+
|               Title|             Genre|       GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|          Table|Language|         dt_inclusao|mes|ano|dia|
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+
|Knights of the Zo...|            Action|            action|2021-03-16|                 TBA|            0|             0|       TBA|        0|        0|Pending|     1|          Anime| English|2021-03-16T21:20:...|-03| 16| 20|
|       Kengan Ashura|            Action|            action|2021-03-16|                 TBA|    

In [175]:
# Questão 03 - #Remover linhas duplicadas e trocar o resultado das linhas que tiverem a coluna "Seasons" de "TBA" para "a ser anunciado".

print(f'Dataframe before drop duplicates: {df_02.toPandas().shape}')
df_03 = df_02.dropDuplicates()
print(f'Dataframe after drop duplicates {df_02.toPandas().shape}')
df_03 = df_02.withColumn("Seasons", when(col("Seasons") == "TBA", "a ser anunciado").otherwise(col("Seasons")))
print(f'Number rows "A ser anunciado:" {df_02.filter(col("Seasons") == "a ser anunciado").count()}')
df_03.show()

Dataframe before drop duplicates: (358, 18)
Dataframe after drop duplicates (358, 18)
Number rows "A ser anunciado:" 0
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+
|               Title|             Genre|       GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|          Table|Language|         dt_inclusao|mes|ano|dia|
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+
|Knights of the Zo...|            Action|            action|2021-03-16|     a ser anunciado|            0|             0|       TBA|        0|        0|Pending|     1|          Anime| English|2021-03-16T

In [176]:
# Questão 04 - Criar uma coluna nova chamada "Data de Alteração" e dentro dela um timestamp.

df_04 = df_03.withColumn('Data de Alteração',current_timestamp())
df_04.show()

+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+--------------------+
|               Title|             Genre|       GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|          Table|Language|         dt_inclusao|mes|ano|dia|   Data de Alteração|
+--------------------+------------------+------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+---------------+--------+--------------------+---+---+---+--------------------+
|Knights of the Zo...|            Action|            action|2021-03-16|     a ser anunciado|            0|             0|       TBA|        0|        0|Pending|     1|          Anime| English|2021-03-16T21:20:...|-03| 16| 20|2022-11-08 02:03:...|
|       Keng

In [177]:
df_04.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: date (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- dia: string (nullable = true)
 |-- Data de Alteração: timestamp (nullable = false)



In [178]:
# Questão 05 - Trocar os nomes das colunas de inglês para português, exemplo: "Title" para "Título" (com acentuação).

df_05 = df_04.withColumnRenamed("Title", "Título") \
            .withColumnRenamed("Genre","Gênero") \
            .withColumnRenamed("GenreLabels", "Rótulos do Gênero") \
            .withColumnRenamed("Premiere", "Pré-estreia") \
            .withColumnRenamed("Seasons", "Temporadas") \
            .withColumnRenamed("SeasonsParsed", "Temporadas Analisadas") \
            .withColumnRenamed("EpisodesParsed", "Episódios Analisados") \
            .withColumnRenamed("Length", "Tamanho") \
            .withColumnRenamed("MinLength", "Tamanho Mínimo") \
            .withColumnRenamed("MaxLength", "Tamanho Máximo") \
            .withColumnRenamed("Status", "Status") \
              .withColumnRenamed("Active", "Ativo") \
            .withColumnRenamed("Table", "Tabela") \
            .withColumnRenamed("Language", "Linguagem") \
            .withColumnRenamed("dt_inclusao", "Data de Inclusão") 

In [179]:
df_05.show()

+--------------------+------------------+------------------+-----------+--------------------+---------------------+--------------------+----------+--------------+--------------+-------+-----+---------------+---------+--------------------+---+---+---+--------------------+
|              Título|            Gênero| Rótulos do Gênero|Pré-estreia|          Temporadas|Temporadas Analisadas|Episódios Analisados|   Tamanho|Tamanho Mínimo|Tamanho Máximo| Status|Ativo|         Tabela|Linguagem|    Data de Inclusão|mes|ano|dia|   Data de Alteração|
+--------------------+------------------+------------------+-----------+--------------------+---------------------+--------------------+----------+--------------+--------------+-------+-----+---------------+---------+--------------------+---+---+---+--------------------+
|Knights of the Zo...|            Action|            action| 2021-03-16|     a ser anunciado|                    0|                   0|       TBA|             0|             0|Pending

In [180]:
df_05.printSchema()

root
 |-- Título: string (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- Rótulos do Gênero: string (nullable = true)
 |-- Pré-estreia: date (nullable = true)
 |-- Temporadas: string (nullable = true)
 |-- Temporadas Analisadas: long (nullable = true)
 |-- Episódios Analisados: long (nullable = true)
 |-- Tamanho: string (nullable = true)
 |-- Tamanho Mínimo: long (nullable = true)
 |-- Tamanho Máximo: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Ativo: long (nullable = true)
 |-- Tabela: string (nullable = true)
 |-- Linguagem: string (nullable = true)
 |-- Data de Inclusão: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- dia: string (nullable = true)
 |-- Data de Alteração: timestamp (nullable = false)



In [ ]:
# Questão 06 - Testar e verificar se existe algum erro de processamento do spark e identificar onde pode ter ocorrido o erro.


# - Entrar no prompt de comando (cmd) e executar o comando a seguir: spark-submit analise_pyspark.py

In [185]:
# Questão 07 - Criar apenas 1 . csv com as seguintes colunas que foram nomeadas anteriormente "Title, Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as colunas devem estar em português com header e separadas por ";".


df_05.select(
    "Título",
    "Gênero",
    "Temporadas",
    "Pré-estreia",
    "Linguagem",
    "Ativo",
    "Status",
    "Data de Inclusão",
    "Data de Alteração"
).repartition(1).write.csv("export.csv", sep=";", header=True, encoding='utf-8')


In [ ]:
# Questão 08 - Inserir esse . csv dentro de um bucket do AWS s3

In [ ]:
import boto3

# A chave foi mascarada por questões de segurança

ACCESS_ID = "AXXXXXXXXN7I5HIZ"
ACCESS_KEY = "98ICVax0WAa9QXCVXCVn+CDSDFS65QY2h3A53v5SveXtG"

s3 = boto3.resource("s3", aws_access_key_id=ACCESS_ID, aws_secret_access_key=ACCESS_KEY)

In [ ]:
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
from glob import glob

data = open(glob("export.csv/*.csv")[0], "rb")
s3.Bucket("Confitec").put_object(Key="export.csv", Body=data)